# Desafio 7 - Dupla 09 - Heurística

**Integrantes:**

* Lucas Hideki Takeuchi Okamura NUSP: 9274315

* Thales Arantes Kerche Nunes NUSP: 10769372

**Objetivo**

* Juntar os algoritmos NEH, LS e IG e então realizar uma análise de variância (ANOVA) em R para verificar as influências entre as variáveis de temperatura e d.

In [1]:
import xlwings as xw
import numpy as np

## 1. Definindo a Class Job

In [2]:
class Job:
    def __init__(self,i,p):
        self.i=i     # número do job, pela ordem de chegada
        self.p=p     # processing times [p1,p2]
        self.C=[]     # completion times [c1,c2]
        self.psum = np.sum(self.p)     # sum of values of p

## 2. Definindo Funções

### Funções Auxiliares

**Encontra a Temperatura (IG)**

In [3]:
def Temperature(T, jobs, m):
    n = len(jobs)
    temperature = T * np.sum([job.psum for job in jobs])/(10* n * m)
    
    return temperature

**Encontrar melhor permutação**

In [4]:
def findBestPerm(jobs, job_insert):
    bestCmax = np.inf
    
    for j in range(len(jobs) + 1):
        jobs.insert(j, job_insert)
        Cmax_test = calcCmax(m, jobs)
        if Cmax_test < bestCmax:
            bestPos = j
            bestCmax = Cmax_test
        jobs.pop(j)
    jobs.insert(bestPos, job_insert)
                
    return jobs, bestCmax

**Função para o cálculo dos tempos de conclusão dos jobs em cada uma das m máquinas**

$p_{(j)k}:$ processing time do job da posição j na máquina k

$c_{(j)k}:$ completion time do job da posição j na máquina k

$c_{(1)1} = p_{(1)1}$

$c_{(1)k}=c_{(1),k−1}+p_{(1)k}$, onde    $k=2,...,m$

$c_{(j)1}=c_{(j−1)1}+p_{(j)1}$, onde $j=2,...,n$

$c_{(j)k}=max\left\{{c_{(j),k−1},c_{(j−1)k}}\right\} +p_{(j)k}$, onde $j=2,...,n$ e $k=2,...,m$

In [5]:
def calcCmax(m,jobs):     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]
    
    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]
        
    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]
            
    j=0
    for job in jobs:
        job.C=c[j]
        j+=1
    
    return c[-1][-1]

**Entrada de Dados**

In [6]:
def leInst(i):
    plan = wb1.sheets[i]
    n=int(plan.range('B6').value)
    m=int(plan.range('B5').value)
    tab=plan.range('B11').expand('table').value
    jobs=[]
    for j in range(n):
        p=tab[j]
        jobs.append(Job(j,p))
        
    #print()
    #print('#','p')
    #for job in jobs:
    #    print(job.i, job.p)
    
    return jobs, m, n

**Saída de resultados**

In [7]:
def gravaSched(instances, table, m, n, d, T, k):
    
    plan=wb2.sheets[0]
    
    plan.name='DoE Desafio 7'
    plan.range('A1').value=["DoE Desafio 7"]
    plan.range('A2').value=["Experimento Fatorial Completo"]
    plan.range('A4').value=["Instâncias - Blocos", "", "i"] + list(np.arange(instances))
    plan.range('A5').value=["d e T - Tratamentos", " ", "d"] + d
    plan.range('A6').value=["k - repetições", " ", "T"] + T
    plan.range('D7').value=["k", k, "(repetições)"]
    plan.range('D8').value=["N", instances * len(d) * len(T) + k, "(experimentos)"]
    plan.range('A10').value=["#", "i", "d", "T", "k", "Cmax"]
    plan.range('A11').value = table

    #print()
    #print('#','p','C')
    #for job in jobs:
    #    print(job.i, job.p, job.C)
    #print('Cmax:',Cmax)

### 2.2. Funções dos algoritmos

**Algoritmo NEH**

In [8]:
def schedNEH(jobs, m):
    jobs.sort(key = lambda job: job.psum, reverse = True)
    n = len(jobs)
    
    # Realizando a primeira iteração
    jobsNEH1 = [jobs[0], jobs[1]]
    Cmax1 = calcCmax(m, jobsNEH1)
    
    jobsNEH2 = [jobs[1], jobs[0]]
    Cmax2 = calcCmax(m, jobsNEH2)
    
    if Cmax1 < Cmax2:
        jobsNEH = jobsNEH1
    else:
        jobsNEH = jobsNEH2
    
    for i in range(2, n):
        bestCmax = float('inf')
        for j in range(i+1):
            NEHcopy = jobsNEH[:]
            NEHcopy.insert(j, jobs[i])
            Cmax = calcCmax(m, NEHcopy)
            if Cmax < bestCmax:
                bestCmax = Cmax
                bestNEH = NEHcopy
        jobsNEH = bestNEH
                
    bestCmax = calcCmax(m, jobsNEH)
    
    return jobsNEH, bestCmax

**Algoritmo LS**

In [9]:
def schedLS(jobs, m, Cmax):
    bestCmax = Cmax
    better = True
    n = len(jobs)
    jobs_final = jobs[:]
    while better:
        better = False
        js = np.arange(n)
        # np.random.shuffle(js)
        for j1 in js:
            job1 = jobs_final.pop(j1)
            bestPos = j1
            for j2 in range(1, n):
                jobs_final.insert(j2, job1)
                Cmax_test = calcCmax(m, jobs_final)
                if Cmax_test < bestCmax:
                    bestPos = j2
                    better = True
                    bestCmax = Cmax_test
                jobs_final.pop(j2)
            jobs_final.insert(bestPos, job1)
            
    return jobs_final, bestCmax

**Algoritmo IG**

In [10]:
def schedIG(jobs, m, Cmax, iter_lim, T, d):    
    # Calculando soma das somas de valores p
    temperature = Temperature(T, jobs, m)
    
    best_jobs = jobs[:]
    
    n = len(jobs)
    
    for lim in range(iter_lim):
        jobs1 = best_jobs[:]
        job_pop = np.random.choice(jobs1, size = d, replace = False)
        
        # Removendo jobs selecionados da lista original
        for job in job_pop:
            jobs1.remove(job)
        
        for job in job_pop:
            jobs1, testCmax = findBestPerm(jobs1, job)
            
        if testCmax < calcCmax(m, best_jobs):
            best_jobs = jobs1[:]
            
        elif temperature != 0. and np.random.random_sample() <= np.exp(-testCmax - calcCmax(m, best_jobs)) / temperature:
            best_jobs = jobs1[:]
            
    return best_jobs, calcCmax(m, best_jobs)

## 3. Testando com Dados do Excel

**Instância de teste menor**

In [11]:
ps = [[16, 18, 12],
      [14, 10, 11],
      [13, 20, 15],
      [19, 15, 19],
      [15, 16, 16]]

jobs_test = []
n = len(ps)
m = len(ps[0])

for i in range(n):
    jobs_test.append(Job(i, ps[i]))
    
T = 0.5
d = 3
    
jobs_test, Cmax = schedNEH(jobs_test, m)
jobs_test, Cmax1 = schedLS(jobs_test, m, Cmax)
jobs_test, Cmax2 = schedIG(jobs_test, m, Cmax1,iter_lim = 100, T = 0.5, d = 3)

print('#','p','C')
for job in jobs_test:
    print(job.i, job.p, job.C)
print('Cmax:',Cmax)

# p C
2 [13, 20, 15] [13, 33, 48]
3 [19, 15, 19] [32, 48, 67]
4 [15, 16, 16] [47, 64, 83]
0 [16, 18, 12] [63, 82, 95]
1 [14, 10, 11] [77, 92, 106]
Cmax: 106


**Instâncias de teste Excel**

In [12]:
wb1 = xw.Book('Desafio 7 - Dupla 09 - Instâncias.xlsx')
wb2 = xw.Book()
I = wb1.sheets.count     # número de instância é igual ao número de planilhas na pasta
d_values = [2, 3, 5, 8]
T_values = [0, 0.1, 0.2, 0.5]
contador = 1
rep = 3
table = []

for i in range(I):
    for d in d_values:
        for T in T_values:
            for k in range(rep):
                print()
                print(i+1)
                jobs, m, n = leInst(i)
                jobs, Cmax = schedNEH(jobs, m)
                jobs, Cmax1 = schedLS(jobs, m, Cmax)
                jobs, Cmax2 = schedIG(jobs, m, Cmax1, iter_lim = 100, T = T, d = d)
                line = [contador, i + 1, d, T, k + 1, Cmax2]
                table.append(line)
                contador += 1
                print("i = {} ; d = {} ; T = {} ; k = {} ; Cmax = {}".format(i + 1, d, T, k + 1, Cmax2))
    
gravaSched(I, table, m, n, d_values, T_values, rep)

wb2.save('Desafio 7 - Dupla 09 - Output.xlsx')


1


<ipython-input-10-37df076d6955>:23: RuntimeWarning: invalid value encountered in double_scalars
  elif np.random.random_sample() <= np.exp(-testCmax - calcCmax(m, best_jobs)) / temperature:


i = 1 ; d = 2 ; T = 0 ; k = 1 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0 ; k = 2 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0 ; k = 3 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.1 ; k = 1 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.1 ; k = 2 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.1 ; k = 3 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.2 ; k = 1 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.2 ; k = 2 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.2 ; k = 3 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.5 ; k = 1 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.5 ; k = 2 ; Cmax = 1207.0

1
i = 1 ; d = 2 ; T = 0.5 ; k = 3 ; Cmax = 1207.0

1
i = 1 ; d = 3 ; T = 0 ; k = 1 ; Cmax = 1188.0

1
i = 1 ; d = 3 ; T = 0 ; k = 2 ; Cmax = 1207.0

1
i = 1 ; d = 3 ; T = 0 ; k = 3 ; Cmax = 1188.0

1
i = 1 ; d = 3 ; T = 0.1 ; k = 1 ; Cmax = 1207.0

1
i = 1 ; d = 3 ; T = 0.1 ; k = 2 ; Cmax = 1207.0

1
i = 1 ; d = 3 ; T = 0.1 ; k = 3 ; Cmax = 1207.0

1
i = 1 ; d = 3 ; T = 0.2 ; k = 1 ; Cmax = 1188.0

1
i = 1 ; d = 3 ; T = 0.2 ; k = 2 ; Cmax = 12

i = 4 ; d = 3 ; T = 0.2 ; k = 2 ; Cmax = 1334.0

4
i = 4 ; d = 3 ; T = 0.2 ; k = 3 ; Cmax = 1334.0

4
i = 4 ; d = 3 ; T = 0.5 ; k = 1 ; Cmax = 1334.0

4
i = 4 ; d = 3 ; T = 0.5 ; k = 2 ; Cmax = 1327.0

4
i = 4 ; d = 3 ; T = 0.5 ; k = 3 ; Cmax = 1334.0

4
i = 4 ; d = 5 ; T = 0 ; k = 1 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0 ; k = 2 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0 ; k = 3 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.1 ; k = 1 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.1 ; k = 2 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.1 ; k = 3 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.2 ; k = 1 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.2 ; k = 2 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.2 ; k = 3 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.5 ; k = 1 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.5 ; k = 2 ; Cmax = 1327.0

4
i = 4 ; d = 5 ; T = 0.5 ; k = 3 ; Cmax = 1327.0

4
i = 4 ; d = 8 ; T = 0 ; k = 1 ; Cmax = 1327.0

4
i = 4 ; d = 8 ; T = 0 ; k = 2 ; Cmax = 1327.0

4
i = 4 ; d = 8 ; T = 0 ; k = 3 ; Cmax = 13